In [1]:
from logging import config
import os
import datetime
from typing import Optional, Tuple
import json

os.environ["WANDB_NOTEBOOK_NAME"] = "Tianshow_Centralized_Training"

import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer, PrioritizedVectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy, RainbowPolicy
from tianshou.trainer import OffpolicyTrainer
from torch.utils.tensorboard import SummaryWriter

from pettingzoo.sisl import pursuit_v4

from TaskAllocation.RL_Policies.MultiHead_SISL import MultiHead_SISL
from TaskAllocation.RL_Policies.DNN_SISL import DNN_SISL
from TaskAllocation.RL_Policies.CNN_SISL import CNN_SISL

from TaskAllocation.RL_Policies.Custom_Classes import CustomNet
from TaskAllocation.RL_Policies.Custom_Classes import CustomCollector
from TaskAllocation.RL_Policies.Custom_Classes import CustomParallelToAECWrapper

from tianshou.utils import WandbLogger


#from tianshou_DQN import train
model  =  "CNN_SISL" #"MultiHead_SISL" 
test_num  =  "_SISL_NOV12_Emb128"
policyModel  =  "DQN"

train_env_num = 10
test_env_num = 10

name = model + test_num

load_policy_name = f'policy_CNN_SISL_SISL_NOV12_Emb128.pth'
save_policy_name = f'policy_{name}'
policy_path = "dqn_SISL"

same_policy = True
load_model = False

# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
log_name = name + str(now)

log_path = os.path.join('./', "Logs", "dqn_sisl", log_name)

SISL_Config = {
    "max_cycles": 500,         # default: 500
    "x_size": 16,              # default: 16
    "y_size": 16,              # default: 16
    "shared_reward": False,     # default: True
    "n_evaders": 30,           # default: 30
    "n_pursuers": 8,           # default: 10
    "obs_range": 7,            # default: 7
    "n_catch": 2,              # default: 2
    "freeze_evaders": False,   # default: False
    "tag_reward": 0.01,        # default: 0.01
    "catch_reward": 5.0,       # default: 5.0
    "urgency_reward": -0.1,    # default: -0.1
    "surround": True,         # default: True
    "constraint_window": 1.0   # default: 1.0
}

max_cycles = SISL_Config["max_cycles"]
n_agents = SISL_Config["n_pursuers"]

dqn_params = {"discount_factor": 0.98, 
              "estimation_step": 20, 
              "target_update_freq": 1000,#max_cycles * n_agents,
              "optminizer": "Adam",
              "lr": 0.00016 }

trainer_params = {"max_epoch": 500,
                  "step_per_epoch": 20000,#5 * (150 * n_agents),
                  "step_per_collect": 400,# * (10 * n_agents),
                  "episode_per_test": 20,
                  "batch_size" : 32 * n_agents,
                  "update_per_step": 1 / 50, #Only run after close a Collect (run many times as necessary to meet the value)
                  "tn_eps_max": 0.1,
                  "ts_eps_max": 0.01,
                  }

runConfig = dqn_params
runConfig.update(trainer_params) 
runConfig.update(SISL_Config)

model_load_path = os.path.join(policy_path, load_policy_name)  
model_save_path = os.path.join(policy_path, save_policy_name)        
os.makedirs(os.path.join(policy_path), exist_ok=True)  
os.makedirs(os.path.join(log_path), exist_ok=True)

def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
    policy_load_path = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    
    env = _get_env()       
    agent_observation_space = env.observation_space
   
    action_shape = env.action_space
    
    device="cuda" if torch.cuda.is_available() else "cpu"          
    
    if agent_learn is None:      
        
        if model == "MultiHead_SISL":
            net = MultiHead_SISL(
                obs_shape=agent_observation_space,                
                action_shape=5,                
                device="cuda" if torch.cuda.is_available() else "cpu"
                
            ).to(device)

        if model == "DNN_SISL":
            net = DNN_SISL(
                obs_shape=agent_observation_space,                
                action_shape=5,                
                device="cuda" if torch.cuda.is_available() else "cpu"
                
            ).to(device)

        if model == "CNN_SISL":
            net = CNN_SISL(
                obs_shape=agent_observation_space.shape,                
                action_shape=5,                
                device="cuda" if torch.cuda.is_available() else "cpu"
                
            ).to(device)
           

        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=dqn_params["lr"], weight_decay=0.0, amsgrad= True )                
    
        if policyModel == "DQN":
            agent_learn = DQNPolicy(
                model=net,
                optim=optim,
                action_space = action_shape,
                discount_factor= dqn_params["discount_factor"],
                estimation_step=dqn_params["estimation_step"],
                target_update_freq=dqn_params["target_update_freq"],
                reward_normalization = False,
                clip_loss_grad = False 
            ) 
                     

        if policyModel == "Rainbow":
            agent_learn = RainbowPolicy(
                model=net.to(device),
                optim=optim,
                action_space = action_shape,
                num_atoms= 5,
                discount_factor= dqn_params["discount_factor"],
                estimation_step=dqn_params["estimation_step"],
                target_update_freq=dqn_params["target_update_freq"],
            ) 
         
 
        if load_model is True:
            # Load the saved checkpoint             
            agent_learn.load_state_dict(torch.load(model_load_path))
            print(f'Loaded-> {model_load_path}')
                   
        #print(env.agents)
        #agents = [agent_learn for _ in range(len(env.agents))]
        
        agents = [agent_learn for _ in range(len(env.agents))]

        
    policy = MultiAgentPolicyManager(policies = agents, env=env)  
        
    return policy, optim, env.agents

def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""   
    # env_paralell = MultiUAVEnv()  
    # env = pursuit_v4.env()
    env = pursuit_v4.env(
            max_cycles=SISL_Config["max_cycles"],
            x_size=SISL_Config["x_size"],
            y_size=SISL_Config["y_size"],
            shared_reward=SISL_Config["shared_reward"],
            n_evaders=SISL_Config["n_evaders"],
            n_pursuers=SISL_Config["n_pursuers"],
            obs_range=SISL_Config["obs_range"],
            n_catch=SISL_Config["n_catch"],
            freeze_evaders=SISL_Config["freeze_evaders"],
            tag_reward=SISL_Config["tag_reward"],
            catch_reward=SISL_Config["catch_reward"],
            urgency_reward=SISL_Config["urgency_reward"],
            surround=SISL_Config["surround"],
            constraint_window=SISL_Config["constraint_window"]
        )

    
    #env = parallel_to_aec_wrapper(env_paralell)    
    # env = CustomParallelToAECWrapper(env_paralell)
    
    return PettingZooEnv(env)

    """This function is needed to provide callables for DummyVectorEnv."""   
    # env_paralell = MultiUAVEnv()  
    env = pursuit_v4.env(
        x_size = 5,
        y_size = 5,
        max_cycles = 100,
        shared_reward = True,
        n_evaders = 10,
        n_pursuers = 4,
        obs_range = 5,
        n_catch = 2,
        surround=False,
        freeze_evaders = False,
        render_mode= None)#"human" if test else None)  
    #env = parallel_to_aec_wrapper(env_paralell)    
    # env = CustomParallelToAECWrapper(env_paralell)
    
    return PettingZooEnv(env)

print(json.dumps(runConfig, indent=4))


{
    "discount_factor": 0.98,
    "estimation_step": 20,
    "target_update_freq": 1000,
    "optminizer": "Adam",
    "lr": 0.00016,
    "max_epoch": 500,
    "step_per_epoch": 20000,
    "step_per_collect": 400,
    "episode_per_test": 20,
    "batch_size": 256,
    "update_per_step": 0.02,
    "tn_eps_max": 0.1,
    "ts_eps_max": 0.01,
    "max_cycles": 500,
    "x_size": 16,
    "y_size": 16,
    "shared_reward": false,
    "n_evaders": 30,
    "n_pursuers": 8,
    "obs_range": 7,
    "n_catch": 2,
    "freeze_evaders": false,
    "tag_reward": 0.01,
    "catch_reward": 5.0,
    "urgency_reward": -0.1,
    "surround": true,
    "constraint_window": 1.0
}


In [2]:
if __name__ == "__main__":
                        
    torch.set_grad_enabled(True) 
   
    # ======== Step 1: Environment setup =========
    train_envs = DummyVectorEnv([_get_env for _ in range(train_env_num)])
    test_envs = DummyVectorEnv([_get_env for _ in range(test_env_num)]) 

    # seed
    seed = 0
    np.random.seed(seed)
    
    torch.manual_seed(seed)

    train_envs.seed(seed)
    test_envs.seed(seed)

    # ======== Step 2: Agent setup =========
    policy, optim, agents = _get_agents()    

    # ======== Step 3: Collector setup =========
    train_collector = Collector(
        policy,
        train_envs,
        VectorReplayBuffer(300_000, len(train_envs)),
        # PrioritizedVectorReplayBuffer( 300_000, len(train_envs), alpha=0.6, beta=0.4) , 
        #ListReplayBuffer(100000)       
        exploration_noise=True             
    )
    test_collector = Collector(policy, test_envs, exploration_noise=True)
     
    print("Buffer Warming Up ")    
    for i in range(1):#int(trainer_params['batch_size'] / (300 * 10 ) )):
        
        train_collector.collect(n_episode=train_env_num)#,random=True) #trainer_params['batch_size'] * train_env_num))
        #train_collector.collect(n_step=300 * 10)
        print(".", end="") 
    
    len_buffer = len(train_collector.buffer) / (SISL_Config["max_cycles"] * SISL_Config["n_pursuers"])
    print("\nBuffer Lenght: ", len_buffer ) 
    
    info = { "Buffer"  : "ReplayBuffer", " Warmup_ep" : len_buffer}
    # ======== tensorboard logging setup =========                       
    logger = WandbLogger(
        train_interval = runConfig["max_cycles"] * runConfig["n_pursuers"] ,
        test_interval = 1,#runConfig["max_cycles"] * runConfig["n_pursuers"],
        update_interval = runConfig["max_cycles"],
        save_interval = 1,
        write_flush = True,
        project = "SISL_Eval01",
        name = log_name,
        entity = None,
        run_id = log_name,
        config = runConfig,
        monitor_gym = True )
    
    writer = SummaryWriter(log_path)    
    writer.add_text("args", str(runConfig))    
    logger.load(writer)
    
    # ======== Step 4: Callback functions setup =========
    def save_best_fn(policy):                
        
        torch.save(policy.policies[agents[0]].state_dict(), model_save_path + ".pth")
        print("Best Saved")
        

    def stop_fn(mean_rewards):
        return mean_rewards >= 99999939.0

    def train_fn(epoch, env_step):
        epsilon = trainer_params['tn_eps_max'] - (trainer_params['tn_eps_max'] - trainer_params['tn_eps_max']/100)*(epoch/trainer_params['max_epoch'])          
        if same_policy:
            policy.policies[agents[0]].set_eps(epsilon)
        else:
            policy.policies['R_agent0'].set_eps(epsilon)
            policy.policies['F_agent0'].set_eps(epsilon)
        
        # if env_step % 500 == 0:
            # logger.write("train/env_step", env_step, {"train/eps": eps})


    def test_fn(epoch, env_step):
        epsilon = trainer_params['ts_eps_max']#0.01#max(0.001, 0.1 - epoch * 0.001)
        if same_policy:
            policy.policies[agents[0]].set_eps(epsilon)
        else:
            policy.policies['R_agent0'].set_eps(epsilon)
            policy.policies['F_agent0'].set_eps(epsilon)

        
    def reward_metric(rews):       
        
        
        return np.sum(rews, axis = 1)


    # #Define the hook function
    # def register_activation_hook(module, input, output, layer_name, writer, global_step_holder):
    #     #print(f"Hook executed for {layer_name} at step {global_step_holder[0]}")
    #     if isinstance(output, tuple):
    #         output = output[0]  # If the output is a tuple, use the first element
    #     writer.add_histogram(f"activations/{layer_name}", output, global_step_holder[0])

    # # ======== Step 5: Run the trainer =========
    offPolicyTrainer = OffpolicyTrainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,        
        max_epoch=trainer_params['max_epoch'],
        step_per_epoch=trainer_params['step_per_epoch'],
        step_per_collect=trainer_params['step_per_collect'],        
        episode_per_test= trainer_params['episode_per_test'],
        batch_size=trainer_params['batch_size'],
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=trainer_params['update_per_step'],
        logger=logger,
        test_in_train=True,
        reward_metric=reward_metric,
        show_progress = True 
               
        )
    
    result = offPolicyTrainer.run()
    writer.close()
    # return result, policy.policies[agents[1]]
    print(f"\n==========Result==========\n{result}")
    print("\n(the trained policy can be accessed via policy.policies[agents[0]])")

d:\Python310\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Buffer Warming Up 


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find Tianshow_Centralized_Training.


.
Buffer Lenght:  10.0


wandb: Currently logged in as: andrekuros. Use `wandb login --relogin` to force relogin
d:\Python310\lib\site-packages\wandb\sdk\lib\ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


...

Epoch #371: 20001it [00:57, 349.08it/s, env_step=7420000, len=4000, n/ep=0, n/st=400, pursuer_0/loss=1.314, pursuer_1/loss=1.496, pursuer_2/loss=1.491, pursuer_3/loss=1.759, pursuer_4/loss=1.345, pursuer_5/loss=1.685, pursuer_6/loss=1.606, pursuer_7/loss=1.590, rew=71.27]                            


Epoch #371: test_reward: 270.570187 ± 177.922711, best_reward: 567.282688 ± 144.595916 in #298


Epoch #372: 20001it [00:59, 338.75it/s, env_step=7440000, len=2616, n/ep=0, n/st=400, pursuer_0/loss=1.410, pursuer_1/loss=1.555, pursuer_2/loss=1.589, pursuer_3/loss=1.643, pursuer_4/loss=1.455, pursuer_5/loss=1.532, pursuer_6/loss=1.682, pursuer_7/loss=1.634, rew=393.40]                           


Epoch #372: test_reward: 458.098500 ± 165.884837, best_reward: 567.282688 ± 144.595916 in #298


Epoch #373:  86%|########6 | 17200/20000 [00:49<00:08, 339.56it/s, env_step=7456800, len=1416, n/ep=0, n/st=400, pursuer_0/loss=1.417, pursuer_1/loss=1.453, pursuer_2/loss=1.598, pursuer_3/loss=1.479, pursuer_4/loss=1.642, pursuer_5/loss=1.557, pursuer_6/loss=1.758, pursuer_7/loss=1.718, rew=367.84]

In [ ]:

from typing import Optional, Tuple
import os
import numpy as np
import torch
from tianshou.env import DummyVectorEnv

import torch

from pettingzoo.sisl import pursuit_v4
from TaskAllocation.RL_Policies.MultiHead_SISL import MultiHead_SISL

import os
import datetime
from typing import Optional, Tuple

import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer, PrioritizedVectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy, RainbowPolicy
from tianshou.trainer import OffpolicyTrainer
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
from TaskAllocation.RL_Policies.CNN_SISL import CNN_SISL

from pettingzoo.sisl import pursuit_v4

from TaskAllocation.RL_Policies.MultiHead_SISL import MultiHead_SISL
from TaskAllocation.RL_Policies.DNN_SISL import DNN_SISL


from TaskAllocation.RL_Policies.Custom_Classes import CustomCollector

#from tianshou_DQN import train
model = "CNN_SISL" #"MultiHead_SISL" 
test_num = "_SISL_NOV12_Emb128"
policyModel = "DQN"

train_env_num = 10
test_env_num = 10

name = model + test_num

load_policy_name = f'policy_CNN_SISL_SISL_NOV12_Emb128_last.pth'
save_policy_name = f'policy_{name}'
policy_path = "dqn_Custom"

same_policy = True

load_model = False

size = [16,16]
n_agents = 8
n_targes = 30
max_cycles = 500
obs_range = 7

# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
log_name = name + str(now)

log_path = os.path.join('./', "Logs", "dqn", log_name)

dqn_params = {"discount_factor": 0.99, 
              "estimation_step": 1, 
              "target_update_freq": 800 * 4,
              "optminizer": "Adam",
              "lr": 1e-4 }

trainer_params = {"max_epoch": 500,
                  "step_per_epoch": 10 * (100 * 10 * 4),
                  "step_per_collect": 20 * (10 * 4),
                  "episode_per_test": 10,
                  "batch_size" : 4 * 32,
                  "update_per_step": 1 / 20, #Only run after close a Collect (run many times as necessary to meet the value)
                  "tn_eps_max": 0.95,
                  "ts_eps_max": 0.0,
                  }


Run_Data = f'''{name}  
Loaded_Model: {load_policy_name if load_model else "no"}  
log_path: {log_path}  
train/test_env_num: {train_env_num} / {test_env_num}  
model: {model}  
dqn_params: {dqn_params}  
trainer_params: {trainer_params} 
single_policy: {same_policy}

--------- Env ------------  
'''
# Rewards Only Final Quality and SQuality
# F_Rew / 20 > lre
# random_init_pos      : {config_default.random_init_pos}
# max_time_steps       : {config_default.max_time_steps}
# simulation_frame_rate: {config_default.simulation_frame_rate}
# Agents               : {config_default.agents}
# tasks                : {config_default.tasks}
# random_init_pos      : {config_default.random_init_pos} 
# threats              : {config_default.threats_list}
# seed                 : {config_default.fixed_seed}
# '''

model_load_path = os.path.join(policy_path, load_policy_name)  
model_save_path = os.path.join(policy_path, save_policy_name)        
os.makedirs(os.path.join(policy_path), exist_ok=True)  
os.makedirs(os.path.join(log_path), exist_ok=True)

def _get_env_eval():
    """This function is needed to provide callables for DummyVectorEnv."""   
    # env_paralell = MultiUAVEnv()  
    env = pursuit_v4.env(
        max_cycles=500,#max_cycles, 
        x_size=size[0], 
        y_size=size[1], 
        shared_reward=True, 
        n_evaders=n_targes,
        n_pursuers=n_agents,
        obs_range=obs_range, 
        n_catch=2, 
        freeze_evaders=False, 
        tag_reward=0.01,
        catch_reward=5.0, 
        urgency_reward=-0.1, 
        surround=True, 
        constraint_window=1.0,
        render_mode = None)#"human")
    #env = parallel_to_aec_wrapper(env_paralell)    
    # env = CustomParallelToAECWrapper(env_paralell)
    
    return PettingZooEnv(env)

def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
    policy_load_path = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    
    env = _get_env_eval()       
    agent_observation_space = env.observation_space
   
    action_shape = env.action_space
    
    device="cuda" if torch.cuda.is_available() else "cpu"          
    
    if agent_learn is None:      
        
        if model == "MultiHead_SISL":
            net = MultiHead_SISL(
                obs_shape=agent_observation_space,                
                action_shape=5,                
                device="cuda" if torch.cuda.is_available() else "cpu"
                
            ).to(device)

        if model == "DNN_SISL":
            net = DNN_SISL(
                obs_shape=agent_observation_space,                
                action_shape=5,                
                device="cuda" if torch.cuda.is_available() else "cpu"
                
            ).to(device)

        if model == "CNN_SISL":
            net = CNN_SISL(
                obs_shape=agent_observation_space.shape,                
                action_shape=5,                
                device="cuda" if torch.cuda.is_available() else "cpu"
                
            ).to(device)
           

        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=dqn_params["lr"], weight_decay=0.0, amsgrad= True )                
    
        if policyModel == "DQN":
            agent_learn = DQNPolicy(
                model=net,
                optim=optim,
                action_space = action_shape,
                discount_factor= dqn_params["discount_factor"],
                estimation_step=dqn_params["estimation_step"],
                target_update_freq=dqn_params["target_update_freq"],
                reward_normalization = False,
                clip_loss_grad = False 
            ) 
                     

        if policyModel == "Rainbow":
            agent_learn = RainbowPolicy(
                model=net.to(device),
                optim=optim,
                num_atoms= 31,
                discount_factor= dqn_params["discount_factor"],
                estimation_step=dqn_params["estimation_step"],
                target_update_freq=dqn_params["target_update_freq"],
            ) 
         
 
        if load_model is True:
            # Load the saved checkpoint             
            agent_learn.load_state_dict(torch.load(model_load_path))
            print(f'Loaded-> {model_load_path}')
                   
        #print(env.agents)
        #agents = [agent_learn for _ in range(len(env.agents))]
        
        agents = [agent_learn for _ in range(len(env.agents))]

        
    policy = MultiAgentPolicyManager(policies = agents, env=env)  
        
    return policy, optim, env.agents


# Create a new instance of the policy with the same architecture as the saved policy
name = 'policy_CNN_SISL_SISL_NOV12_Emb128.pth' 
load_policy_name = f'policy_{name}'



log_path = os.path.join('./', "Logs", "dqn", name)

policy, optim, _ = _get_agents()
model_save_path = os.path.join("dqn_SISL", save_policy_name)        

# Load the saved checkpoint
policy_test = policy.policies['pursuer_0']
policy_test.load_state_dict(torch.load(model_save_path + ".pth" ))

envs = DummyVectorEnv([_get_env_eval for _ in range(1)])
#policy_test.eval()
policy_test.set_eps(0.05)

#collector = CustomCollector(policy.policies['agent0'], envs, exploration_noise=True)
#collector = CustomCollector(policy_test, envs, exploration_noise=False)
collector = CustomCollector(policy, envs, exploration_noise=True)

#results = collector.collect(n_episode=1)
results = collector.collect(n_episode=1)#, render=0.1,)#, gym_reset_kwargs={'seed' :2})
results

In [ ]:
results['rews']
print(np.mean(results['rews'][results['rews'] > -10]))


#create a function  to print a histogram of the results['rews']
import matplotlib.pyplot as plt
plt.hist(results['rews'][results['rews'] > -10], bins=100)
plt.show()
